In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [23]:
tops=pd.read_csv(r'M:\jonahtops.csv', names=['api', 'formation', 'depth', 'ssdepth'])
small=tops.head()

In [10]:
n_wells = tops.api.unique().shape[0]
n_formations = tops.formation.unique().shape[0]

print('Number of wells: {}'.format(n_wells))
print('Number of formations: {}'.format(n_formations))
print('Sparsity: {:4.3f}%'.format(float(tops.shape[0]) / float(n_users*n_items) * 100))

Number of wells: 2885
Number of formations: 456
Sparsity: 1.149%


In [24]:
small

,api,formation,depth,ssdepth
0,3521913,WASATCH,0,7281
1,3521913,LANCE,9285,-2004
2,3521913,LANCE - OVER PRESSURE,9760,-2479
3,3521913,JONAH,11227,-3946
4,3521913,YELLOW POINT,11793,-4512


In [36]:
from scipy.sparse import *
from scipy import *

In [50]:
csr_matrix((array([0,-1,-2,-3,-4]), (array([1,1,1,1,1]), array([0,1,2,3,4])))).todense()

matrix([[ 0,  0,  0,  0,  0],
        [ 0, -1, -2, -3, -4]], dtype=int32)

In [38]:
row = array([0,0,1,2,2,2])
col = array([0,2,2,0,1,2])
data = array([1,2,3,4,5,6])
csr_matrix( (data,(row,col)), shape=(3,3) ).todense()

matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]], dtype=int32)

In [51]:
csr_matrix(small)

TypeError: no supported conversion for types: (dtype('O'),)

In [62]:
labels, names=pd.factorize(small['formation'])
small['fms']=labels

C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [72]:
sparse=csr_matrix(np.array(small[['api', 'ssdepth', 'fms']]).astype(int))

In [76]:
sparse

<5x3 sparse matrix of type '<class 'numpy.int32'>'
	with 14 stored elements in Compressed Sparse Row format>